In [1]:
  from google.colab import drive #connect to google drive
drive.mount('Mydrive')

Mounted at Mydrive


In [2]:
pwd

'/content'

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import shutil
import os

In [4]:
train_path="/content/Mydrive/MyDrive/Colab Notebooks/AML_project/train"
test_path="/content/Mydrive/MyDrive/Colab Notebooks/AML_project/test"
#IMG_SIZE = 224 # MobilenetV2
RESOLUTION = 224
PATCH_SIZE = 16

num_classes = 100
input_shape = (224, 224, 3)

In [5]:
#declare plant species dictionary
plant_species_dict = {}
with open('/content/Mydrive/MyDrive/Colab Notebooks/AML_project/list/species_list.txt') as txt_file:
    lines =  [x.strip() for x in txt_file.readlines()]
    plant_class = [x.split('; ')[0] for x in lines]
    plant_species = [x.split('; ')[1] for x in lines]

for i in range(len(plant_class)):
    plant_species_dict[plant_class[i]] = plant_species[i]

#plant_species_dict

In [6]:
from os import listdir
from matplotlib import image
import tensorflow as tf
import numpy as np

In [7]:
x_train = tf.keras.utils.image_dataset_from_directory(train_path + "/herbarium",
                                                            shuffle=True,
                                                            batch_size=64,
                                                            image_size=(224, 224),
                                                            )
                                                      
x_test = tf.keras.utils.image_dataset_from_directory(test_path,
                                                                 shuffle=True,
                                                                 batch_size=64,
                                                                 image_size=(224, 224)
                                                                 )



Found 3018 files belonging to 100 classes.
Found 1933 files belonging to 100 classes.


In [8]:
ViT = "https://tfhub.dev/sayakpaul/vit_b8_fe/1"

In [9]:
#get weights
import tensorflow_hub as hub
ViT_net_layers = hub.KerasLayer(ViT, input_shape=(224,224,3))

In [10]:
#set layers trainable to false
ViT_net_layers.trainable = False

In [11]:
#data augmentation
data_augmentation=tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal_and_vertical'),
  tf.keras.layers.RandomRotation(0.1),
  tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1)
])

In [12]:
def create_vit_classifier():
    inputs = layers.Input(shape=(224,224,3))
    augmented = data_augmentation(inputs)
    vit = ViT_net_layers(augmented)
    #features = layers.Dropout(0.3)(vit)
    features = layers.Dense(128)(vit)
    features = layers.Dense(64)(features)
    logits = layers.Dense(100, kernel_regularizer=tf.keras.regularizers.l2(0.001))(features)
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [13]:
#set up model
vit_classifier = create_vit_classifier()

In [14]:
#declare top 1 accuracy metric
top1 = tf.keras.metrics.TopKCategoricalAccuracy(k=1)

In [15]:
#compile model
vit_classifier.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy', top1])
vit_classifier.summary() # prints model summary

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 768)               85807872  
                                                                 
 dense (Dense)               (None, 128)               98432     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 100)               6500      
                                                                 
Total params: 85,921,060
Trainable params: 113,188
Non-traina

In [ ]:
#train the model
model_fit = vit_classifier.fit(x_train, epochs=20, validation_data=x_test)       

Epoch 1/20


48/48 [==============================] - 1548s 29s/step - loss: 5.7673 - accuracy: 0.0232 - top_k_categorical_accuracy: 3.3135e-04 - val_loss: 4.8701 - val_accuracy: 0.0285 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 2/20
48/48 [==============================] - 329s 7s/step - loss: 4.6067 - accuracy: 0.0563 - top_k_categorical_accuracy: 0.0242 - val_loss: 4.7470 - val_accuracy: 0.0378 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 3/20
48/48 [==============================] - 328s 7s/step - loss: 4.4229 - accuracy: 0.0736 - top_k_categorical_accuracy: 0.0305 - val_loss: 4.8545 - val_accuracy: 0.0528 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 4/20
48/48 [==============================] - 329s 7s/step - loss: 4.2430 - accuracy: 0.0924 - top_k_categorical_accuracy: 0.0232 - val_loss: 4.7520 - val_accuracy: 0.0378 - val_top_k_categorical_accuracy: 0.0000e+00
Epoch 5/20
48/48 [==============================] - 329s 7s/step - loss: 4.1312 - accuracy: 0.1183 - top_k_catego

In [ ]:
train_acc = vit_classifier.evaluate(x_train)
test_acc = vit_classifier.evaluate(x_test )

print(f'Train accuracy: {train_acc[1]*100}')
print(f'Test accuracy: {test_acc[1]*100}')

In [ ]:
#set layers trainable to true
ViT_net_layers.trainable = True

In [ ]:
#compile model
vit_classifier.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy', top1])
vit_classifier.summary() # prints model summary

In [ ]:
#train the model
model_fit = vit_classifier.fit(x_train, epochs=30, validation_data=x_test)       

In [ ]:
#print accuracy of training and test set
_, train_acc = vit_classifier.evaluate(training_generator)
_, test_acc = vit_classifier.evaluate(validation_generator )

print(f'\nTrain accuracy: {train_acc:.0%}')
print(f'Test accuracy: {test_acc:.0%}')